In [ ]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.utils.plot import (
    plot_parameter_stat_profile,
    plot_spectrum_fp,
    plot_spectrum_model
)
from gammapy.modeling.models import Models
import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u

In [ ]:
config_file = "/path/to/Config.yaml"

In [ ]:
config = AsgardpyConfig()

In [ ]:
config_main = config.read(config_file)
only_3d_config = config.read(config_file)
only_1d_config = config.read(config_file)

In [ ]:
for g in config_main.general:
    print(g)

# Steps mentioned in the main config file

In [ ]:
config_main.general.steps

# Target source information

In [ ]:
for c in config_main.target:
    print(c)

# Datasets informations

In [ ]:
for cc in config_main.dataset3d:
    print(cc)

In [ ]:
for cc in config_main.dataset1d:
    print(cc)

In [ ]:
for cc in config_main.fit_params:
    print(cc)

## Update the Target parameters for only 1D analysis, in the config

In [ ]:
print(only_1d_config.target.components.spectral.parameters[0])
only_1d_config.target.components.spectral.parameters[0].value = 1e-9
only_1d_config.target.components.spectral.parameters[0].error = 1e-10
print(only_1d_config.target.components.spectral.parameters[0])

In [ ]:
print(only_1d_config.target.components.spectral.parameters[1])
only_1d_config.target.components.spectral.parameters[1].value = 0.07
print(only_1d_config.target.components.spectral.parameters[1])

In [ ]:
print(only_1d_config.target.components.spectral.parameters[2])
only_1d_config.target.components.spectral.parameters[2].value = 2.0
print(only_1d_config.target.components.spectral.parameters[2])

In [ ]:
analysis = AsgardpyAnalysis(config_main)

In [ ]:
only_3d_analysis = AsgardpyAnalysis(only_3d_config)

In [ ]:
only_1d_analysis = AsgardpyAnalysis(only_1d_config)

In [ ]:
analysis

# Run all the steps

In [ ]:
%%time
analysis.run()

In [ ]:
%%time
only_3d_analysis.run(["datasets-3d", "fit", "flux-points"])

In [ ]:
%%time
only_1d_analysis.run(["datasets-1d", "fit", "flux-points"])

# Get the final products

In [ ]:
for d in analysis.datasets:
    print(d)

In [ ]:
for d in analysis.datasets:
    print(d.name)

In [ ]:
analysis.fit_result

In [ ]:
print(only_3d_analysis.fit_result)
print(only_1d_analysis.fit_result)

In [ ]:
print(analysis.final_model)

In [ ]:
print(only_3d_analysis.final_model)

In [ ]:
print(only_1d_analysis.datasets.models)

In [ ]:
analysis.final_model.parameters.to_table()

In [ ]:
analysis.final_model.covariance.get_subcovariance(
    analysis.final_model.parameters.free_parameters
).plot_correlation()

In [ ]:
print(analysis.final_model[config_main.target.source_name])

In [ ]:
if len(analysis.final_model[config_main.target.source_name].parameters)> 3:
    nrow = int(np.sqrt(len(analysis.final_model[config_main.target.source_name].parameters)))
    ncol = int(np.ceil(len(analysis.final_model[config_main.target.source_name].parameters)/nrow))
else:
    ncol = len(analysis.final_model[config_main.target.source_name].parameters)
    nrow = 1
print(ncol, nrow)

In [ ]:
%%time
plt.figure(figsize=(ncol*6, nrow*5))
total_stat = analysis.fit_result.total_stat

for i, par in enumerate(analysis.final_model[config_main.target.source_name].parameters.free_parameters):
    par.scan_n_values = 30
    
    idx = analysis.final_model[config_main.target.source_name].parameters.index(par)
    name = Models(analysis.final_model[config_main.target.source_name]).parameters_unique_names[idx]
    
    profile = analysis.fit.stat_profile(datasets=analysis.datasets, parameter=par)
    
    plt.subplot(nrow, ncol, i+1)
    plt.plot(profile[f"{name}_scan"], profile["stat_scan"] - total_stat)
    plt.xlabel(f"{par.name} {par.unit}")
    plt.ylabel("Delta TS")
    plt.grid()
    if par.value != 0:
        plt.title(f"{par.name}: {par.value:.2e} +/-{par.error:.2e}\n(Relative Error +/-{par.error/par.value:.2e})")
    else:
        plt.title(f"{par.name}: {par.value:.2e} +/-{par.error:.2e}")
plt.tight_layout()

In [ ]:
%%time
# Only 3D Dataset
plt.figure(figsize=(ncol*6, nrow*5))
total_stat = only_3d_analysis.fit_result.total_stat

for i, par in enumerate(only_3d_analysis.final_model[config_main.target.source_name].parameters.free_parameters):
    par.scan_n_values = 30
    
    idx = only_3d_analysis.final_model[config_main.target.source_name].parameters.index(par)
    name = Models(only_3d_analysis.final_model[config_main.target.source_name]).parameters_unique_names[idx]
    
    profile = only_3d_analysis.fit.stat_profile(datasets=only_3d_analysis.datasets, parameter=par)
    
    plt.subplot(nrow, ncol, i+1)
    plt.plot(profile[f"{name}_scan"], profile["stat_scan"] - total_stat)
    plt.xlabel(f"{par.name} {par.unit}")
    plt.ylabel("Delta TS")
    plt.grid()
    if par.value != 0:
        plt.title(f"{par.name}: {par.value:.2e} +/-{par.error:.2e}\n(Relative Error +/-{par.error/par.value:.2e})")
    else:
        plt.title(f"{par.name}: {par.value:.2e} +/-{par.error:.2e}")
plt.tight_layout()

In [ ]:
%%time
# Only 1D Dataset
plt.figure(figsize=(ncol*6, nrow*5))
total_stat = only_1d_analysis.fit_result.total_stat

for i, par in enumerate(Models(only_1d_analysis.datasets.models).parameters.free_parameters):
    par.scan_n_values = 30

    idx = only_1d_analysis.datasets.models.parameters.index(par)
    name = Models(only_1d_analysis.datasets.models).parameters_unique_names[idx]
    
    profile = only_1d_analysis.fit.stat_profile(datasets=only_1d_analysis.datasets, parameter=par)
    
    plt.subplot(nrow, ncol, i+1)
    plt.plot(profile[f"{name}_scan"], profile["stat_scan"] - total_stat)
    plt.xlabel(f"{par.name} {par.unit}")
    plt.ylabel("Delta TS")
    plt.grid()
    if par.value != 0:
        plt.title(f"{par.name}: {par.value:.2e} +/-{par.error:.2e}\n(Relative Error +/-{par.error/par.value:.2e})")
    else:
        plt.title(f"{par.name}: {par.value:.2e} +/-{par.error:.2e}")
plt.tight_layout()

In [ ]:
for fp in analysis.flux_points:
    print(fp)

In [ ]:
analysis.flux_points[0].to_table(formatted=True, sed_type="e2dnde")

In [ ]:
analysis.flux_points[1].to_table(formatted=True, sed_type="e2dnde")

In [ ]:
# Checking TS profiles of only Joint-Fit
analysis.flux_points[0].plot_ts_profiles(sed_type="e2dnde")
analysis.flux_points[0].plot(sed_type="e2dnde", color="orange")

plt.grid(which='both')
plt.title('TS Profiles')

In [ ]:
analysis.flux_points[1].plot_ts_profiles(sed_type="e2dnde")
analysis.flux_points[1].plot(sed_type="e2dnde", color="orange")

plt.grid(which='both')
plt.title('TS Profiles')

In [ ]:
e_plot_min = 100 * u.MeV
e_plot_max = 10 * u.TeV

sed_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [e_plot_min, e_plot_max]
}
sed_plot_kwargs = {
    "sed_type": "e2dnde",
}

In [ ]:
only_3d_analysis.flux_points[0].plot(sed_type="e2dnde", color="orange", label="Only Fermi-LAT")
only_1d_analysis.flux_points[0].plot(sed_type="e2dnde", color="green", label="Only LST-1")

analysis.flux_points[0].plot(sed_type="e2dnde", color="red", label="Joint-Fit Fermi-LAT")
analysis.flux_points[1].plot(sed_type="e2dnde", color="blue", label="Joint-Fit LST-1")

sed_kwargs["label"] = ""
analysis.final_model[0].spectral_model.plot_error(**sed_kwargs)
sed_kwargs["label"] = "Joint Fit Observed model"
analysis.final_model[0].spectral_model.plot(**sed_kwargs)
sed_kwargs["label"] = "Joint Fit Intrinsic model"
analysis.final_model[0].spectral_model.model1.plot(**sed_kwargs)

plt.grid(which="both")
plt.legend(bbox_to_anchor=(1,1))
plt.xlim(e_plot_min, e_plot_max)

In [ ]:
# Only Models
sed_kwargs["label"] = ""
analysis.final_model[0].spectral_model.plot_error(**sed_kwargs)
sed_kwargs["label"] = "Joint Fit Observed model"
analysis.final_model[0].spectral_model.plot(**sed_kwargs)

sed_kwargs["label"] = ""
only_3d_analysis.final_model[0].spectral_model.plot_error(**sed_kwargs)
sed_kwargs["label"] = "Only Fermi-LAT Observed model"
only_3d_analysis.final_model[0].spectral_model.plot(**sed_kwargs)

sed_kwargs["label"] = ""
only_1d_analysis.datasets.models[0].spectral_model.plot_error(**sed_kwargs)
sed_kwargs["label"] = "Only LST-1 Observed model"
only_1d_analysis.datasets.models[0].spectral_model.plot(**sed_kwargs)

plt.grid(which="both")
plt.legend()
plt.xlim(e_plot_min, e_plot_max)